In [2]:
from __future__ import division
import six
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
import re
import numpy as np
import csv
import pandas as pd
import sys

In [30]:
GOOGLE_APPLICATION_CREDENTIALS="Research//google_credentials.json"

In [21]:
def clean_tweet(tweet): 
    ''' 
    Utility function to clean tweet text by removing links, special characters 
    using simple regex statements. 
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [22]:
def get_tweets(handle): 
    ''' 
    Main function to fetch tweets and parse them. 
    '''
    # eppty list to store parsed tweets 
    with open('Tweets//' + handle + '_tweets.csv', 'r') as f:
        reader = csv.reader(f)
        tweets = list(reader)
          
    fetched_tweets = []
    for i in np.arange(1,len(tweets)):
        fetched_tweets.append(tweets[i][2])
        
    return fetched_tweets

In [33]:
tweets = get_tweets('ewarren')

In [41]:
def word_in_text(word, text):
    word = word.lower()
    text = text.lower()
    match = re.search(word, text)
    if match:
        return True
    return False

def word_basket_in_text(word_list, text):
    status = True
    for word in word_list:
        if type(word)==list:
            list_status = False
            for list_word in word:
                list_status = list_status or word_in_text(list_word, text)
            status = status and list_status
        else: status = status and word_in_text(word, text)
    return status

In [35]:
Healthcare = [u'care','affordable care act', 'medicare', 'obamacare',  'benefits', 'opioid', 'abortion', 'medicaid', 'single payer', 'medication', 'prescription', 'drugs', 'hospital', 'health', 'care']
Terrorism = [u'al', u'terror', u'sanctions', u'drone', u'oil', u'gulf', u'intelligence', u'ISIS', u'military', u'nuclear']
Economy = [u'china', u'tariff', u'mexico', u'fed', u'sec', u'interest', u'rate', u'immigration', u'debt', u'unemployment', u'growth', u'inflation', u'trade war', u'dollar', u'gdp', u'g20', u'recession', u'stock', u'market']
lgbtq = [u'gay', u'lesbian', u'pride', u'community', u'lgbt', u'same-sex', u'marriage', u'orientation', u'parade', u'stonewall', u'riot', u'civil', u'homosexual', u'queer', u'conversion']

In [42]:
# Final List

# Healthcare
affordable_care = ['health', ['affordable', 'obamacare', 'medicare', 'care']]
medicare = ['medicare'] 
safe_abortion = ['abortion', ['safe', 'legal']]
abortion_ban = ['abortion', ['ban', 'restrictions', 'extreme']]

# Education
education_for_all = ['education', ['afford', 'free']]
student_loans = ['student', ['cancel', 'loans', 'debt']]

# Immigration
immigration = ['immigration', ['policy', 'policies', 'mexico', 'illegal', 'undocumented', 'border']]
asylum = ['border', ['detention', 'families', 'family', 'asylum']]

In [48]:
BJ_healthcare = []
for tweet in tweets:
    if word_basket_in_text(affordable_care, tweet):
        BJ_healthcare.append(clean_tweet(tweet))

In [49]:
BJ_healthcare = list(set(BJ_healthcare))

In [45]:
def implicit():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client()

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

In [46]:
client = language.LanguageServiceClient.from_service_account_json(GOOGLE_APPLICATION_CREDENTIALS)

def sentiment_analyzer(list_of_tweets, list_of_keywords): 
    for text in list_of_tweets:
        if isinstance(text, six.binary_type):
            text = text.decode('utf-8')

        document = types.Document(
            content=text.encode('utf-8'),
            type=enums.Document.Type.PLAIN_TEXT)

        # Detect and send native Python encoding to receive correct word offsets.
        encoding = enums.EncodingType.UTF32
        if sys.maxunicode == 65535:
            encoding = enums.EncodingType.UTF16

        result = client.analyze_entity_sentiment(document, encoding)
        print(text)
        for entity in result.entities:
            entity_name = entity.name
            if entity_name in list_of_keywords:
                print(u'Name: "{}"'.format(entity.name))
                for mention in entity.mentions:
                    print(u'  Begin Offset : {}'.format(mention.text.begin_offset))
                    print(u'  Content : {}'.format(mention.text.content))
                    print(u'  Magnitude : {}'.format(mention.sentiment.magnitude))
                    print(u'  Sentiment : {}'.format(mention.sentiment.score))
                    print(u'  Type : {}'.format(mention.type))
                print(u'Salience: {}'.format(entity.salience))
                print(u'Sentiment: {}\n'.format(entity.sentiment))
        print('--------')

In [51]:
sentiment_analyzer(BJ_healthcare, ['affordable care act', 'healthcare', 'affordable', 'obamacare', 'medicare', 'MedicareForAll'])

Another sick attack on transgender Americans coming from the White House All LGBTQ people should be able to access high quality gender affirming health care Period
--------
First Congress should create federal statutory rights that parallel Roe s protections We must prohibit states from interfering in the ability of a health professional to provide medical care including abortion care amp from interfering in the ability of a patient to access care
--------
Today is the 2019 deadline to enroll in health care for many states We must keep fighting to protect health care for millions of Americans Tell your loved ones to sign up TODAY
--------
When the Republicans say they want to repeal health care for 24 million people today they need to explain this
--------
Working families in the Central Valley need someone who ll fight for them in Congress amp that s harder He fights for health care good jobs small businesses amp more But we need to fight for him in this neck and neck House race
-----

The time to act is now Get into the fight run for office and make these right wing Republican lawmakers face the consequences of their actions Here s my plan to protect reproductive health care and StopTheBans
--------
Right here in Puerto Rico our federal government is taking money off the island in order to send it to Wall Street and cut services health care services amp schools I talked about this amp more on last night Watch our interview
--------
Yes I would support government run insurance Health care is a basic human right and we fight for basic human rights We need MedicareForAll DemDebate
--------
National Grid locked out Massachusetts workers amp families like Kerrie s have lost their health care for months We need to EndTheLockout amp give workers the health care amp pensions they deserve I m fighting with National Grid workers all the way
--------
Wow s strong grassroots campaign turned VA05 a Republican leaning district into a toss up We re THIS close to sending her to Con

Workers have a right to fight for higher wages health care and pensions and I m with and all the way
--------
My friend is a hero not only to me or his son Carl but to millions of people whose health care he s fought for I hope you ll join Ady amp beahero too
--------
Health care is a basic human right and we fight for basic human rights When I lead the Democratic Party we will be a party that says We re the wealthiest nation in the history of the world so yes we can afford MedicareForAll
Name: "MedicareForAll"
  Begin Offset : 210
  Content : MedicareForAll
  Magnitude : 0.30000001192092896
  Sentiment : 0.30000001192092896
  Type : 1
Salience: 0.0076313684694468975
Sentiment: magnitude: 0.30000001192092896
score: 0.30000001192092896


--------
It s great to be back in Wisconsin to support amp my friend We need Tammy back in the Senate to keep fighting for health care amp Social Security Get out and vote Wisconsin
--------
On I talked about how Trump s got the trifecta someone to over